# Performance e Operações Paralelas

In [3]:
using Pkg

using BenchmarkTools
using LoopVectorization
using Plots
using StaticArrays
using ThreadsX

using CUDA # há vá?!?!

using Random: seed!, shuffle
using Statistics: mean

# Random Seed
seed!(123)

# Printar cores no terminal
using ANSIColoredPrinters

# Checklist basico para _performance_

1. Arrumar instabilidade de tipo
2. Usar variáveis locais ao invés de gloabais
3. Deixar tudo imutável se possível
4. Desativar checagem de índice em operações com `Array`
5. Ativar suporte SIMD em todos os loops `for` (Single Instruction Multiple Data)

## Instabilidade de tipo

> Tipo de saída de uma função é __imprevisível__ a partir dos tipos de entradas. Em particular, isso significa que o tipo de saída __pode variar__ dependendo dos valores das entradas.

In [4]:
function positivo(x)
    if x > 0
        return x
    else
        return 0
    end
end

positivo (generic function with 1 method)

> função com untabilidade de tipo >>> `x` é o que?

`@code_warntype` avalia a função como um argumento e prenta um _Abstract Syntax Tree_ (AST)

In [5]:
@code_warntype positivo(-3.4)

Variables
  #self#::Core.Const(positivo)
  x::Float64

Body::Union{Float64, Int64}
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


> onde ficou vermelho, tem algum problema de tipo

### Arrumando a instabilidade de tipo

Anotar os tipos!

In [6]:
function positivo_stabel(x::AbstractFloat)
    if x > 0
        return x 
    else
        return 0.0
    end
end

positivo_stabel (generic function with 1 method)

In [7]:
function positivo_stabel(x::Integer)
    if x > 0
        return x 
    else
        return 0
    end
end

positivo_stabel (generic function with 2 methods)

In [8]:
@code_warntype positivo_stabel(-3.4)

Variables
  #self#::Core.Const(positivo_stabel)
  x::Float64

Body::Float64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0.0


In [9]:
@code_warntype positivo_stabel(-3)

Variables
  #self#::Core.Const(positivo_stabel)
  x::Int64

Body::Int64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      return 0


### Pq é importante anotar os tipos?

In [11]:
x = rand(1_000);

In [13]:
@benchmark positivo.($x)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.930 μs …  1.357 ms  ┊ GC (min … max):  0.00% … 99.49%
 Time  (median):     3.980 μs              ┊ GC (median):     0.00%
 Time  (mean ± σ):   7.814 μs ± 37.978 μs  ┊ GC (mean ± σ):  17.16% ±  3.58%

  ▄▁ ▆█▅▃▃▃▃▂     ▄▃▄▅▃▂▁                                    ▁
  █████████████▇▇██████████████▇▇▇▇█▇▆▆▄▆▅▆▆▅▄▅▄▄▅▄▄▄▄▂▃▃▃▃▅ █
  1.93 μs      Histogram: log(frequency) by time     26.5 μs <

 Memory estimate: 8.22 KiB, allocs estimate: 7.

In [14]:
@benchmark positivo_stabel.($x)

BenchmarkTools.Trial: 10000 samples with 186 evaluations.
 Range (min … max):  803.763 ns … 348.528 μs  ┊ GC (min … max):  0.00% … 96.99%
 Time  (median):       1.267 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     2.478 μs ±   6.365 μs  ┊ GC (mean ± σ):  20.15% ± 13.30%

  ▇█▆▄▃▃▂▁▂▂▁▁                                                  ▁
  ███████████████▇▇▆▅▆▆▇▆▅▅▅▅▅▄▆▆▆▆▅▅▆▆▆▆▅▅▅▄▅▅▄▅▅▅▅▄▅▅▅▃▃▃▃▄▃▃ █
  804 ns        Histogram: log(frequency) by time       21.8 μs <

 Memory estimate: 7.94 KiB, allocs estimate: 1.

### Dicas!

Dar preferência para usar __tipos abstrtatos__ ao invés de tipos concretos.

`AbstractFlosa` >>> `Float64` or `Float32`  
`Integer` >>> `Int128` or `Int64` or `Int342`

### Tipos paramétricos

São introduzidos com as chaves `{}` e usando palavras-chave `where`

In [16]:
# por exemplo qualquer subtipo de `Real`
subtypes(Real)

8-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 FixedPointNumbers.FixedPoint
 Integer
 Rational
 StatsBase.PValue
 StatsBase.TestStat
 VectorizationBase.AbstractSIMD

In [17]:
function positivo_stable2(x::T) where T <: Real
    if x > 0
        return x
    else
        return 0::T
    end
end

positivo_stable2 (generic function with 1 method)

In [18]:
@code_warntype positivo_stable2(-3.4)

Variables
  #self#::Core.Const(positivo_stable2)
  x::Float64

Body::Float64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─      Core.typeassert(0, $(Expr(:static_parameter, 1)))
└──      Core.Const(:(return %4))


In [19]:
@code_warntype positivo_stable2(-3)

Variables
  #self#::Core.Const(positivo_stable2)
  x::Int64

Body::Int64
1 ─ %1 = (x > 0)::Bool
└──      goto #3 if not %1
2 ─      return x
3 ─ %4 = Core.typeassert(0, $(Expr(:static_parameter, 1)))::Core.Const(0)
└──      return %4
